In [41]:
import re
from bs4 import BeautifulSoup
import requests
from textblob import TextBlob

#Retrieve headlines from webpage using BeautifulSoup. Returns a list of headlines
def headline_retriever(url, element, class_):
    
    r=requests.get(url)
    page = r.content

    soup=BeautifulSoup(page, 'html5lib')
    headlines=soup.find_all(element, class_=class_)
    
    return headlines

#Separates headlines into headlines about liberals, and headlines about conservatives. This is based on whether or 
#not the headlines contain words from the two word banks. 
def headline_separator(headlines): 
    
    liberal_words=['Biden', 'democrat', 'Democrat', 'liberal']
    conservative_words=['Trump', 'GOP', 'republican', 'Republican', 'conservative']
    
    liberal_headlines=[]
    conservative_headlines=[]
    
    for i in range(len(headlines)):
        head = headlines[i].get_text()
        for a in conservative_words:
            if a in head:
                conservative_headlines.append(head)
                break
        for b in liberal_words:
            if b in head:
                liberal_headlines.append(head)
                break
                      
    return conservative_headlines, liberal_headlines

#This function measures sentiment of each headline, and returns a statement on whether or not liberals or conservatives
#recieve more positive headlines
def sentiment_meter(conservative_headlines, liberal_headlines, title_of_news):
    
    #Run each headline through Textblob and obtain the sentiment score
    conservative_sentiment_list=[]
    
    for i in range(len(conservative_headlines)):
        text = conservative_headlines[i]
        text = re.sub("\n", " ", text)
        text = TextBlob(text)
        conservative_sentiment_list.append(text.sentiment.polarity)
        
    liberal_sentiment_list=[]
        
    for i in range(len(liberal_headlines)):
        text = liberal_headlines[i]
        text = re.sub("\n", " ", text)
        text = TextBlob(text)
        liberal_sentiment_list.append(text.sentiment.polarity)
    
    #Count how many sentiments are positive or negative, with regard to the subject of the headline (liberals or 
    #conservatives)
    conservative_good_count=0
    conservative_bad_count=0
    for i in conservative_sentiment_list:
        if i > 0:
            conservative_good_count +=1
        if i < 0:
            conservative_bad_count +=1
            
    conservative_good = conservative_good_count/len(conservative_sentiment_list)
    conservative_bad = conservative_bad_count/len(conservative_sentiment_list)
    
    overall_conservative = conservative_good - conservative_bad
    
    liberal_good_count=0
    liberal_bad_count=0
    for i in liberal_sentiment_list:
        if i > 0:
            liberal_good_count +=1
        if i < 0:
            liberal_bad_count +=1
            
    liberal_good = liberal_good_count/len(liberal_sentiment_list)
    liberal_bad = liberal_bad_count/len(liberal_sentiment_list)
    
    overall_liberal = liberal_good - liberal_bad
    
    #Return a statement showing the difference in positive coverage for liberals vs conservatives
    if overall_conservative > overall_liberal:
        overall = overall_conservative/overall_liberal
        print(title_of_news+' writes '+str(overall)+'% better about conservatives than liberals')
    
    if overall_liberal > overall_conservative:
        overall = overall_liberal/overall_conservative
        print(title_of_news+' writes '+str(overall)+'% better about liberals than conservatives')        

In [42]:
headlines = headline_retriever('https://www.foxnews.com', 'h2', 'title title-color-default')
conservative_headlines, liberal_headlines = headline_separator(headlines)
sentiment_meter(conservative_headlines, liberal_headlines, 'fox news')

fox news writes 1.4999999999999998% better about liberals than conservatives
